In [21]:
import pandas as pd
import numpy as np

import csv
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV,RandomizedSearchCV
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.model_selection import train_test_split
from validation import cross_val, Preprocessing, cross_gen
import math
from sklearn.metrics import roc_auc_score

In [2]:
train = pd.read_csv('train_cleaned.csv',sep = '\t')
test = pd.read_csv('test_cleaned.csv',sep = '\t')

In [3]:
X = train.drop('0',axis=1)
y= train['0']
Xt = test.drop('0',axis=1)
yt = test['0']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.12, random_state=42)

# LogisticRegression

In [5]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [6]:
lr = LogisticRegression()
cv_score = np.mean(cross_val_score(lr,X_train, y_train, scoring='roc_auc',n_jobs=-1))
lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [11]:
y_pred2 = lr.predict_proba(Xt)

In [8]:
y_pred = lr.predict_proba(X_test)

In [9]:
roc_auc_score(y_test, y_pred[:,1] )#0.73486478

0.7312271026155143

In [34]:
submission_2 = pd.DataFrame({'_ID_': yt.index})
submission_2['_VAL_'] = y_pred2[:,1]
submission_2.to_csv('submission_log1234.csv', sep=',', index=False)

In [ ]:
submission_2.head(5)

# RandomForestClassifier

In [10]:
parameters = {'max_features': [4,7, 10, 13], 'min_samples_leaf': [1,3,5,7], 'max_depth': [10,15,20]}
rfc = RandomForestClassifier(n_estimators =100 ,random_state=42, n_jobs=-1)
gcv = GridSearchCV(rfc, parameters, n_jobs=-1, cv=skf, verbose=1,  scoring ='roc_auc')
gcv.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  7.1min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:  9.6min finished


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
       error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_features': [4, 7, 10, 13], 'min_samples_leaf': [1, 3, 5, 7], 'max_depth': [10, 15, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=1)

In [12]:
best_rf = gcv.best_estimator_

In [13]:
y_pred_h = best_rf.predict_proba(X_test)

In [14]:
roc_auc_score(y_test, y_pred_h[:,1] )

0.738161457355585

In [17]:
y_pred_rfc = best_rf.predict_proba(Xt)

In [116]:
col_rfc = []
importances = best_etc.feature_importances_
feat_labels = X_train.columns[0:]
indices = np . argsort(importances)[::-1 ] 
for f in range(X_train .shape[1]):
    if importances[indices[f]] < 0.000011:
        col_rfc.append(feat_labels[indices[f]])
    print ("%2d) %- *s %f " % (f + 1, 30 , feat_labels[indices[f]] , importances[indices[f]]))

 1) 337                            0.028377 
 2) 294                            0.021112 
 3) 86                             0.018942 
 4) 336                            0.018328 
 5) 4                              0.017776 
 6) 144                            0.016509 
 7) 34                             0.016101 
 8) 60                             0.015402 
 9) 65                             0.014247 
10) 214                            0.012226 
11) 91                             0.011393 
12) 332                            0.011266 
13) 324                            0.011051 
14) 96                             0.010529 
15) 334                            0.010457 
16) 36                             0.010260 
17) 88                             0.010243 
18) 325                            0.009739 
19) 323                            0.009696 
20) 331                            0.008997 
21) 282                            0.008960 
22) 344                            0.008522 
23) 118   

IndexError: index 321 is out of bounds for axis 0 with size 321

In [117]:
X_best_rfc = X_train.drop(col_rfc,axis=1)
X_test_best_rfc = X_test.drop(col_rfc,axis=1)
best_rf.fit(X_best_rfc, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=15, max_features=13, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=7, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [118]:
y_pred_hrfc2 = best_rf.predict_proba(X_test_best_rfc)

In [119]:
roc_auc_score(y_test, y_pred_hrfc2[:,1])#0.73568169

0.738287134976187

In [120]:
Xt_best_rfc = Xt.drop(col_rfc,axis=1)
y_pred_rfc2 = best_rf.predict_proba(Xt_best_rfc)

In [121]:
submission_rfc = pd.DataFrame({'_ID_': yt.index})
submission_rfc['_VAL_'] = y_pred_rfc2[:,1]
submission_rfc.to_csv('submission_rf12345.csv', sep=',', index=False)

# BaggingClassifier

In [19]:
from sklearn.ensemble import BaggingClassifier

In [22]:
parameters = {'max_features': [0.5, 0.7, 0.9, 1.], 'max_samples': [0.5, 0.7, 0.9, 1.], "base_estimator__C": [0.005, 0.001, 0.01]}
bg = BaggingClassifier(lr, random_state=42, n_estimators=25)
r_grid_search = RandomizedSearchCV(bg, parameters, scoring ='roc_auc', n_iter=10, cv=skf, random_state=1)
r_grid_search.fit(X_train, y_train)

RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
          error_score='raise',
          estimator=BaggingClassifier(base_estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start..., n_estimators=25, n_jobs=1, oob_score=False,
         random_state=42, verbose=0, warm_start=False),
          fit_params=None, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'max_features': [0.5, 0.7, 0.9, 1.0], 'max_samples': [0.5, 0.7, 0.9, 1.0], 'base_estimator__C': [0.005, 0.001, 0.01]},
          pre_dispatch='2*n_jobs', random_state=1, refit=True,
          return_train_score='warn', scoring='roc_auc', verbose=0)

In [23]:
best_bg = r_grid_search.best_estimator_

In [25]:
y_pred_hbg = best_bg.predict_proba(X_test)

In [26]:
roc_auc_score(y_test, y_pred_hbg[:,1])#0.72539552

0.7271123436983189

In [27]:
y_pred_bg = best_bg.predict_proba(Xt)

In [28]:
submission_bg = pd.DataFrame({'_ID_': yt.index})
submission_bg['_VAL_'] = y_pred_bg[:,1]
submission_bg.to_csv('submission_bg1234.csv', sep=',', index=False)

# ExtraTreesClassifier

In [122]:
parameters = {'max_features': [3, 4, 5, 6, 7], 'min_samples_leaf': [1, 3, 5], 'max_depth': [5,10,15,20]}
etc = ExtraTreesClassifier(n_estimators=100, random_state=42, n_jobs=-1)
etcv = GridSearchCV(etc, parameters, n_jobs=-1, cv=skf, verbose=1, scoring='roc_auc')
etcv.fit(X_train, y_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   45.2s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  8.1min finished


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
       error_score='raise',
       estimator=ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
           oob_score=False, random_state=42, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_features': [3, 4, 5, 6, 7], 'min_samples_leaf': [1, 3, 5], 'max_depth': [5, 10, 15, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=1)

In [123]:
best_etc = etcv.best_estimator_

In [124]:
y_pred_hetc = best_etc.predict_proba(X_test)

In [125]:
roc_auc_score(y_test, y_pred_hetc[:,1])

0.747267563447479

In [46]:
y_pred_etc = best_etc.predict_proba(Xt)

In [126]:
col_etc = []
importances = best_etc.feature_importances_
feat_labels = X_train.columns[0:]
indices = np . argsort(importances)[::-1 ] 
for f in range(X_train .shape[1]):
    if importances[indices[f]] < 0.000011:
        col_etc.append(feat_labels[indices[f]])
    print ("%2d) %- *s %f " % (f + 1, 30 , feat_labels[indices[f]] , importances[indices[f]]))

 1) 295                            0.021380 
 2) 337                            0.020391 
 3) 86                             0.019949 
 4) 338                            0.017908 
 5) 144                            0.017284 
 6) 34                             0.015963 
 7) 335                            0.015866 
 8) 65                             0.015720 
 9) 60                             0.015114 
10) 4                              0.014893 
11) 96                             0.013207 
12) 333                            0.011328 
13) 147                            0.010976 
14) 88                             0.010823 
15) 324                            0.010497 
16) 91                             0.010475 
17) 325                            0.010089 
18) 283                            0.009831 
19) 214                            0.009828 
20) 323                            0.009417 
21) 118                            0.008745 
22) 62                             0.008613 
23) 319   

In [127]:
X_best = X_train.drop(col_etc,axis=1)
best_etc.fit(X_best, y_train)

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=20, max_features=7, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=3, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [128]:
X_test_best = X_test.drop(col_etc,axis=1)


In [129]:
y_pred_hetc2 = best_etc.predict_proba(X_test_best)

In [130]:
roc_auc_score(y_test, y_pred_hetc2[:,1])

0.7516915208622221

In [131]:
Xt_best = Xt.drop(col_etc,axis=1)

In [132]:
y_pred_etc2 = best_etc.predict_proba(Xt_best)

In [133]:
submission_bg = pd.DataFrame({'_ID_': yt.index})
submission_bg['_VAL_'] = y_pred_etc2[:,1]
submission_bg.to_csv('submission_etc12345.csv', sep=',', index=False)